# <center><font color=slate>Object Internals and Custom Attributes</font></center>

## <center>How are <font color=tomato>Python objects represented?</font></center>


In [50]:
class Vector:
    def __init__(self, x, y):
        self._x = x
        self._y = y

    def __repr__(self):
        return "{}({} {})".format(self.__class__.__name__, self._x, self._y)

v = Vector(5, 3)
v

Vector(5 3)

In [51]:
dir(v)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_x',
 '_y']

`__dict__` is a dictionary, contains the names of our objects attributes as keys, and the values of our objects attributes as values

In [52]:
type(v.__dict__), v.__dict__

(dict, {'_x': 5, '_y': 3})

In [53]:
v.__dict__['_x']

5

We can even modify attributes:


In [54]:
v.__dict__['_x'] = 17
v.__dict__['_x']

17

.... and even remove attributes

In [55]:
del v.__dict__['_x']
v.__dict__

{'_y': 3}

We can also test for their existence using the `in` operator

In [56]:
'_x' in v.__dict__, '_y' in v.__dict__

(False, True)

and add new attributes into the dictionary, and thereby into our object


In [57]:
v.__dict__['_z'] = 13
v._z

13

Although all of these direct queries and manipulations of `__dict__` are possible, for the most part, you should prefer to use the built-in functions `getattr`, `hasattr`, `delattr`, and `setattr`

In [58]:
getattr(v, '_y')

3

In [59]:
hasattr(v, '_x')

False

In [60]:
delattr(v, '_z')

In [61]:
setattr(v, '_x', 9)
v

Vector(9 3)

Our vector class, like most vector classes, has hard wired attributes called x and y to store the two components of the vector.
Many problems, though, require us to deal with vectors in different coordinate systems within the same code or perhaps it's just convenient to use a different labeling scheme, such as u and v instead of x and y in a particular context.

In [62]:

class Vector(Vector):
    def __init__(self, **coords):
        self.__dict__.update(coords)

    def __repr__(self):
        return "{}({})".format(
            self.__class__.__name__,
            ', '.join('{k}={v}'.format(
                k = k,
                v = self.__dict__[k])
                      for k in sorted(self.__dict__.keys())))

v = Vector(p=3, q=7)
v

Vector(p=3, q=7)

In [63]:
dir(v)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'p',
 'q']

Our coordinates are now essentially public attributes of our vector objects.
What if we want our vector class to be an immutable value type, so we provide values to the constructor, which can't be subsequently changed?
Ordinarily we will do this by prefixing our attributes with an underscore to signal that they are implementation details, and then provide a property with only a getter to prevent modification

In [64]:
class Vector(Vector):
    def __init__(self, **coords):
        private_coords = {'_' + k: v for k, v in coords.items()}
        self.__dict__.update(private_coords)

    def __repr__(self):
        return "{}({})".format(
            self.__class__.__name__,
            ', '.join('{k}={v}'.format(
                k = k[1:],
                v = self.__dict__[k])
                      for k in sorted(self.__dict__.keys())))
v = Vector(p=3, q=7)
v

Vector(p=3, q=7)

In [65]:
dir(v)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_p',
 '_q']

### <font color=lightGreen>Overriding</font>`__getattr__`
To fake the existence of p for read access, and to do that we need to intercept attribute access before the AttributeError is raised.


In [66]:
class Vector(Vector):
    def __getattr__(self, name):
        private_name = '_' + name
        return getattr(self, private_name)
v = Vector(p=3, q=7)
v

Vector(p=3, q=7)

In [67]:
v.p, v.q

(3, 7)

but still we can create a real attribute p

In [68]:
v.p = 20

In [69]:
v.p, v._p

(20, 3)

### <font color=lightGreen>Overriding</font>`__setattr__`
We can prevent this by overriding `__setattr__` too to intercept attempts to write to our attributes

In [70]:
class Vector(Vector):
    def __setattr__(self, name, value):
        raise AttributeError("Can't set attribute {!r}".format(name))
v = Vector(p=3, q=7)
v

Vector(p=3, q=7)

In [71]:
v.p, v.q

(3, 7)

now attributes cannot be changed

In [72]:
try:
    v.p = 20
except Exception as e:
    print(e.__repr__(), '\n context: ', e.__context__.__repr__())

AttributeError("Can't set attribute 'p'") 
 context:  None


### <font color=lightGreen>Pitfalls with</font>`__setattr__`
Although `__getattr__` works fine with attributes we've carefully faked, such as p,
when we try to access an attribute for which there is no fake support, such as x we get a runtime error, `maximum recursion depth exceeded while calling a Python objec`

In [73]:
try:
    v.x
except Exception as e:
    print(e.__repr__(), '\n context: ', e.__context__.__repr__())

RecursionError('maximum recursion depth exceeded while calling a Python object') 
 context:  None


Instead of invoking getattr, we can just directly return the attribute from `__dict__`.

In [74]:
class Vector(Vector):

    def __getattr__(self, name):
        private_name = '_' + name
        try:
            return self.__dict__[private_name]
        except KeyError:
            raise AttributeError('{!r} object has no attribute {!r}'.format(self.__class__, name))

        return getattr(self, private_name)

v = Vector(p=3, q=7)
v

Vector(p=3, q=7)

In [75]:
v.p, v.q

(3, 7)

In [76]:
try:
    v.x
except Exception as e:
    print(e.__repr__(), '\n context: ', e.__context__.__repr__())

AttributeError("<class '__main__.Vector'> object has no attribute 'x'") 
 context:  KeyError('_x')


In [77]:
try:
    v.x = 5
except Exception as e:
    print(e.__repr__(), '\n context: ', e.__context__.__repr__())


AttributeError("Can't set attribute 'x'") 
 context:  None


### <font color=lightGreen>Overriding</font>`__delattr__()`
Deleting attributes is something that is very rarely seen in Python, although it is possible


In [78]:
delattr(v, '_p')

In [79]:
del v._q

In [80]:
v

Vector()

Although a client is unlikely to attempt this inadvertently, we can prevent it by overriding `__delattr__`

In [81]:
class Vector(Vector):

    def __delattr__(self, name):
        raise AttributeError("Can't delete attribute {!r}".format(name))

v = Vector(p=3, q=7)
v

Vector(p=3, q=7)

In [82]:
try:
    del v.p
except Exception as e:
    print(e.__repr__(), '\n context: ', e.__context__.__repr__())

AttributeError("Can't delete attribute 'p'") 
 context:  None


### <font color=lightGreen>Customizing attribute storage</font>
Store attributes directly in `__dict__`

In [83]:
class ColoredVector(Vector):
        COLOR_INDEXES = ('red', 'green', 'blue')
        def __init__(self, red, green, blue, **coords):
            super().__init__(**coords)
            self.__dict__['_color'] = [red, green, blue]

        def __getattr__(self, name):
            try:
                channel = ColoredVector.COLOR_INDEXES.index(name)
            except ValueError:
                return super().__getattr__(name)
            else:
                return self.__dict__['_color'][channel]

        def __setattr__(self, name, value):
            try:
                channel = ColoredVector.COLOR_INDEXES.index(name)
            except ValueError:
                super().__setattr__(name, value)
            else:
                self.__dict__['_color']['channel'] = value

cv = ColoredVector(red=23, green=44, blue=238, p=9, q=14)
cv

ColoredVector(color=[23, 44, 238], p=9, q=14)

In [84]:
cv.red, cv.blue, cv.green, cv.p, cv.q

(23, 238, 44, 9, 14)

In [85]:
dir(cv)

['COLOR_INDEXES',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_color',
 '_p',
 '_q']

### <font color=lightGreen>Overriding</font>`__delattribute__()`

In [86]:
v.__dict__

{'_p': 3, '_q': 7}

In [87]:
cv.__class__.__dict__

mappingproxy({'__module__': '__main__',
              'COLOR_INDEXES': ('red', 'green', 'blue'),
              '__init__': <function __main__.ColoredVector.__init__(self, red, green, blue, **coords)>,
              '__getattr__': <function __main__.ColoredVector.__getattr__(self, name)>,
              '__setattr__': <function __main__.ColoredVector.__setattr__(self, name, value)>,
              '__doc__': None})

### <font color=lightGreen>Trading Size for Dynamism with Slots</font>
## <center>`__slots__`</center>
Is is a mechanism in Python for reducing memory use
with the getsizeof() from the module sys we can find out the size of an object in memory

In [88]:
import sys
d = {} #empty dictionary
sys.getsizeof(d)

64

In [89]:
class Resistor:

    def __init__(self, resistance_ohms, tolerance_percent, power_watts):
        self._resistance_ohms = resistance_ohms
        self._tolerance_percent = tolerance_percent
        self._power_watts = power_watts

r10 = Resistor(10, 5, 0.25)
sys.getsizeof(r10.__dict__)

104

In [90]:
r10.cost_dollars = 0.02

In [91]:
sys.getsizeof(r10.__dict__)


104

In [92]:
class Resistor():

    __slots__ = ['_resistance_ohms', '_tolerance_percent', '_power_watts']

    def __init__(self, resistance_ohms, tolerance_percent, power_watts):
        self._resistance_ohms = resistance_ohms
        self._tolerance_percent = tolerance_percent
        self._power_watts = power_watts

r10 = Resistor(10, 5, 0.25)
sys.getsizeof(d)

64

There's always a tradeoff, as we can no longer dynamically add attributes to instances of resistor.

In [93]:
try:
    r10.cost_dollars = 0.02
except Exception as e:
    print(e.__repr__(), '\n context: ', e.__context__.__repr__())

AttributeError("'Resistor' object has no attribute 'cost_dollars'") 
 context:  None


This is because the internal structure of resistor no longer contains a dunder dict

In [94]:
try:
    r10.__dict__
except Exception as e:
    print(e.__repr__(), '\n context: ', e.__context__.__repr__())

AttributeError("'Resistor' object has no attribute '__dict__'") 
 context:  None


In [95]:
dir(r10)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_power_watts',
 '_resistance_ohms',
 '_tolerance_percent']